In [ ]:
import os
import random

from astropy.io import fits
from astropy.utils.data import download_file
from astropy.visualization import imshow_norm, ZScaleInterval
from astropy.visualization.stretch import  SqrtStretch
from astropy.time import Time
import matplotlib.pyplot as plt

from pyvo.dal.sia2 import SIA2Service
import pyvo

from lsst.rsp import get_datalink_result, get_siav2_service
from lsst.rsp.service import get_siav2_service
from lsst.rsp.utils import get_pyvo_auth, get_access_token, get_service_url


In [ ]:
# TODO: We shouldn't have to specify the release here probably 
sia_url = get_service_url("sia/dp02")

In [ ]:
# TODO: This will be hidden from the user

import requests
token = get_access_token()

s = requests.Session()
s.headers["Authorization"] = "Bearer " + token
auth = pyvo.auth.authsession.AuthSession()
auth.credentials.set("lsst-token", s)
auth.add_security_method_for_url(sia_url + "/query", "lsst-token")


In [ ]:
# Get an object to query the SIAV2 service with
service = SIA2Service(sia_url, session=auth)

In [ ]:
# Execute a query looking for pictures within a circle
# and show them as a table
t1 = Time("60550.31803461111", format='mjd').to_datetime()
t2 = Time("60550.31838182871", format='mjd').to_datetime()

results = service.search(pos=(55.7467, -32.2862, 0.05), time=[t1,t2])


In [ ]:
results.to_table()

In [ ]:
# Extract the access URL from the result in the first row
result = random.choice(results)
f"Datalink link service url: {result.getdataurl()}"

In [ ]:
# Retrieve the datalink votable document and
# show it in the notebook
dl_result = get_datalink_result(result)
dl_result.to_table().show_in_notebook()

In [ ]:
# Full image of calexp - not a cutout
image_url = dl_result.getrecord(0).get('access_url')
image_url

In [ ]:
# Now let's download the image 
filename = download_file(image_url)
filename

In [ ]:
hdulist = fits.open(image_url)

for hdu in hdulist:
    print(hdu.name)

In [ ]:
# Let's plot the image and see what it looks like...
image = hdulist[1].data

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
im = imshow_norm(image, ax, origin='lower', interval=ZScaleInterval(), stretch=SqrtStretch(), cmap='gray')
fig.colorbar(im[0])